In [3]:
# !pip install scikit-surprise

     |████████████████████████████████| 11.8 MB 5.7 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619429 sha256=308ed86e5e5837ace4d650d698bdedc42f9b5bc9965e967426375d534f30bd63
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [4]:
import surprise

In [5]:
print(surprise.__version__)

1.1.1


In [30]:
from surprise import SVD 
##고유값 분해(행렬을 더 낮은 차원으로 분해)해서 다시 원본으로 살리면서 잠재적인 값을 추론하는 방법
from surprise import Dataset ## SVD를 사용하기 위한 데이터셋 만들어주는 클래스
                            ## 사용자, 아이템, 평점
from surprise import accuracy ## RMSE, MAE
from surprise.model_selection import train_test_split ## 훈련/ 검증 데이터 분류

In [7]:
## 1. 데이터셋을 만들어주자(사용자, 아이템, 평점), 검증/테스트 데이터 분류
data = Dataset.load_builtin('ml-100k')
data

Dataset ml-100k could not be found. Do you want to download it? [Y/n] ㅛ
Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [8]:
trainset, testset = train_test_split(data, test_size= 0.25,random_state= 0)
trainset

In [97]:
trainset

In [87]:
## 2. .SVD() 모델 선정
algo=SVD(n_factors= 20, n_epochs=30, random_state=1)

In [88]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성. 
data_folds = DatasetAutoFolds(ratings_file='ratings_noh.csv', reader=reader)

#전체 데이터를 학습데이터로 생성함. 
trainset = data_folds.build_full_trainset()

In [105]:
data_folds

In [89]:
## 3. 훈련용 데이터로 fit()
algo.fit(trainset)

In [90]:
## 4. 검증용 데이터로 예측
predictions = algo.test(testset)[:5]
predictions

[Prediction(uid=63, iid=2000, r_ui=3.0, est=3.501556983616962, details={'was_impossible': False}),
 Prediction(uid=31, iid=788, r_ui=2.0, est=3.501556983616962, details={'was_impossible': False}),
 Prediction(uid=159, iid=6373, r_ui=4.0, est=3.501556983616962, details={'was_impossible': False}),
 Prediction(uid=105, iid=81564, r_ui=3.0, est=3.501556983616962, details={'was_impossible': False}),
 Prediction(uid=394, iid=480, r_ui=3.0, est=3.501556983616962, details={'was_impossible': False})]

In [91]:
[(pred.uid, pred.iid, pred.est) for pred in predictions]

[(63, 2000, 3.501556983616962),
 (31, 788, 3.501556983616962),
 (159, 6373, 3.501556983616962),
 (105, 81564, 3.501556983616962),
 (394, 480, 3.501556983616962)]

In [17]:
## 5. 정확도 계산 
accuracy.rmse(predictions)

RMSE: 0.9084


0.9084328688416029

In [92]:
## 6. 사용자, 아이템정보 주고 예측 실행
uid =str(196)
iid=str(203)
pred = algo.predict(uid,iid)
pred

Prediction(uid='196', iid='203', r_ui=None, est=3.454142918863094, details={'was_impossible': False})

In [93]:
import pandas as pd 

In [19]:
## csv를 SVD하기 위한 Dataset을 주기 위해서는 
## 1) df <--- csv, 2) Dataset<-- df

In [23]:
ratings = pd.read_csv('/content/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [26]:
from surprise import Reader

In [32]:
reader = Reader(rating_scale=(0.5,5.0))
reader

In [34]:
data = Dataset.load_from_df(ratings[['userId','movieId','rating']],reader)
data

In [35]:
## 데이터 분류, 객체생성, 훈련시키고, 검증해보고, 검증결과 프린트

In [36]:
trainset, testset = train_test_split(data, test_size= 0.25,random_state= 0)
trainset

In [37]:
algo.fit(trainset)

In [42]:
predictions = algo.test(testset)[:5]
predictions

[Prediction(uid=63, iid=2000, r_ui=3.0, est=3.743179978888842, details={'was_impossible': False}),
 Prediction(uid=31, iid=788, r_ui=2.0, est=3.358902550345712, details={'was_impossible': False}),
 Prediction(uid=159, iid=6373, r_ui=4.0, est=2.9888319209900702, details={'was_impossible': False}),
 Prediction(uid=105, iid=81564, r_ui=3.0, est=4.01473604009848, details={'was_impossible': False}),
 Prediction(uid=394, iid=480, r_ui=3.0, est=3.236585914201492, details={'was_impossible': False})]

In [43]:
accuracy.rmse(predictions)

RMSE: 0.9494


0.9494267356925686

In [44]:
uid =str(70)
iid=str(2000)
pred = algo.predict(uid,iid)
pred

Prediction(uid='70', iid='2000', r_ui=None, est=3.5002644558160445, details={'was_impossible': False})

In [46]:
# 영화에 대한 상세 속성 정보 DataFrame로딩
movies = pd.read_csv('/content/movies (1).csv')

# userId=9 의 movieId 데이터 추출하여 movieId=42 데이터가 있는지 확인. 
movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [47]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.50   {'was_impossible': False}


In [50]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [99]:
def get_unseen_surprise(movies, ratings, userId):
  ## 전체영화id 리스트
  total_movies= movies['movieId'].tolist()
  ## 내가 본 영화id 리스트
  seen_movies = ratings[ratings['userId']==userId]['movieId'].tolist()
  ## 추천 대상이 되는 영화리스트 = 영화리스트 - 내가 본 영화 리스트
  unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
  print('전체 영화 리스트 개수 >> ', len(total_movies))
  print('내가 본 영화 리스트 개수 >> ', len(seen_movies))
  print('내가 안본 영화 리스트 개수 >> ', len(unseen_movies))
  return unseen_movies

In [100]:
unseen_movies = get_unseen_surprise(movies,ratings, 9)

전체 영화 리스트 개수 >>  9742
내가 본 영화 리스트 개수 >>  46
내가 안본 영화 리스트 개수 >>  9696


In [101]:
## 안본 영화중에서 평점 예측이 높게 나온 3개를 리스트업하는 함수. 
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n = 5):
  ## 안본 영화리스트를 하나씩 꺼낸다음 평점을 예측하세요.
  predictions = [algo.predict(str(userId),str(movieId)) for movieId in unseen_movies]
  ## 내림차순 정렬
  def sortkey_est(one):
    return one.est
  predictions.sort(key = sortkey_est,reverse=True)
  top_predictions = predictions[:top_n]
  top_movie_ids = [ int(pred.iid) for pred in top_predictions]
  top_movie_rating = [ pred.est for pred in top_predictions]
  top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
  top_movie_preds = list(zip(top_movie_ids, top_movie_rating, top_movie_titles))

  return top_movie_preds

In [102]:
top_predictions = recomm_movie_by_surprise(algo, 9, unseen_movies)
top_predictions

[(750, 4.476281873922615, 'Shawshank Redemption, The (1994)'),
 (1089,
  4.322834963507822,
  'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)'),
 (1208, 4.298289575182489, 'Reservoir Dogs (1992)'),
 (318, 4.297756703770234, 'Apocalypse Now (1979)'),
 (3030, 4.278983600619643, 'Yojimbo (1961)')]

In [155]:
yeon = pd.read_csv('/content/movie_rating.csv')
yeon.head()

,critic,title,rating
0,Jack,Lady,3.0
1,Jack,Snakes,4.0
2,Jack,You Me,3.5
3,Jack,Superman,5.0
4,Jack,The Night,3.0


In [128]:
movie = pd.DataFrame(list(set(yeon.title.values)),columns={'title'})
movie

,title
0,Just My
1,Snakes
2,Superman
3,You Me
4,The Night
5,Lady


In [ ]:
data = Dataset.load_from_df(ratings, reader)
data

In [120]:
trainset,testset=train_test_split(data, test_size=.25, random_state=0)

In [121]:
algo = SVD(n_factors=50, n_epochs= 40,  random_state=0)
algo.fit(trainset) 
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8784


0.8783983314109463

In [135]:
def get_unseen_surprise2(ratings, movies, userId):
    #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['critic']== userId]['title'].tolist()
    print(seen_movies)
    # 모든 영화들의 movieId를 리스트로 생성. 
    total_movies = movies['title'].tolist()
    
    # 모든 영화들의 movieId중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
    unseen_movies= [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:',len(seen_movies), '추천대상 영화수:',len(unseen_movies), \
          '전체 영화수:',len(total_movies))
    
    return unseen_movies


In [139]:
unseen_movie= get_unseen_surprise2(yeon,movie,'Toby')
unseen_movie

['Snakes', 'Superman', 'You Me']
평점 매긴 영화수: 3 추천대상 영화수: 3 전체 영화수: 6


['Just My', 'The Night', 'Lady']

In [154]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(userId, str(movieId)) for movieId in unseen_movies]
    
    def sortkey_est(pred):
        return pred.est
    
    # # sortkey_est( ) 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions= predictions[:top_n]
    
    print(top_predictions)
    
    # # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [ pred.iid for pred in top_predictions]
    top_movie_rating = [ pred.est for pred in top_predictions]
    top_movie_preds = [ (id,  rating) for id,  rating in zip(top_movie_ids, top_movie_rating)]
    
    return top_movie_preds

top_movie_preds = recomm_movie_by_surprise(algo, 'Toby', unseen_movie, top_n=3)

for top_movie in top_movie_preds:
    print(top_movie[0], ":", top_movie[1])

[Prediction(uid='Toby', iid='Just My', r_ui=None, est=3.5002644558160445, details={'was_impossible': False}), Prediction(uid='Toby', iid='The Night', r_ui=None, est=3.5002644558160445, details={'was_impossible': False}), Prediction(uid='Toby', iid='Lady', r_ui=None, est=3.5002644558160445, details={'was_impossible': False})]
Just My : 3.5002644558160445
The Night : 3.5002644558160445
Lady : 3.5002644558160445
